### Imports

In [ ]:
import sys
import os
sys.path.append("..\\")
from src.features import mfcc

In [ ]:
# Define the window size and hop size in seconds
window_size = 0.025  # 25 milliseconds
hop_size = 0.010  # 10 milliseconds

# Calculate win_length based on the window_size and sample_rate
sample_rate = 16000
win_length = int(window_size * sample_rate)

In [ ]:
# Essential directories
root_directory = '..\\data\\raw\\audio_classified\\'
target_dir = '..\\data\\processed\\'
label_to_index = {'Fluent': 0, 'Nonfluent': 1}


## Audio feature extraction

### Load data

In [ ]:
# load & generate MFCC features
dataloader = mfcc.get_dataloader(root_directory, label_to_index)

#### Save processed dataset in pickle file

In [ ]:
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

mfcc.save_mfcc_features(dataloader, target_dir)

### Load processed dataset

In [ ]:
features, labels = mfcc.load_mfcc_features(target_dir)
print(f"Features shape: {features.shape},\n \
      Labels shape: {labels.shape},\n \
      Number of classes: {len(set(labels))},\n \
      Number of samples: {len(features)}")

### Train-test split

In [ ]:
# Imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from src.features.triplets import *

seed = 42

In [ ]:
# Normalize feature matrix
mean = np.mean(features, axis=0)
std = np.std(features, axis=0)
# normalized_features = (all_features - mean) / std
normalized_features = (features - mean) / (std + 1e-8)  # Add a small epsilon to prevent division by zero

# Encode class labels as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training, validation, and test sets
X_train, X_val, Y_train, Y_val = train_test_split(normalized_features, labels_encoded, test_size=0.1, random_state=seed)

print(f" \
      Train set: {X_train.shape} \n \
      Train label: {Y_train.shape} \n \
      Validation set: {X_val.shape} \n \
      Validation label: {Y_val.shape} \n \
")

### Generate Triplets

In [ ]:
# Generate triplets for training
train_triplets, train_labels = generate_triplets(X_train, Y_train)

# NOTE: Size ~ 1.1GB

# For validation & test set | Probably unnecessary
val_triplets, val_labels = generate_triplets(X_val, Y_val)

#### Save Triplets

In [ ]:
# NOTE: Modify functions if I figure out val and test triplets are unnecessary.
save_triplets(
    (
        train_triplets, 
        train_labels,
        val_triplets,
        val_labels,
    ),
    target_dir
)

#### Load Triplets

In [ ]:
# !!Obsolete!! DO NOT RUN!
# NOTE: Looks clumsy with too many variables to unpack
train_triplets, train_labels, val_triplets, val_labels, test_triplets, test_labels = load_triplets()

In [ ]:
from src.data.dataset import MyDataset

In [ ]:
train_dataset = MyDataset(target_dir, "train")

## Initializing Deep Learning network

In [ ]:
# imports
import torch
from src.models import config


In [ ]:
# make config changes here & initialize training loop

#### Train model

In [ ]:
# imports
from src.models import train, test

In [ ]:
train(train_dataset)

### Evaluate

### Test

In [ ]:
test_dataset = MyDataset(target_dir, "test")

In [ ]:
test(test_dataset)

## Facial landmark feature extraction

In [ ]:
# imports
landmark_root_dir = "..\\data\\raw\\AllLandmarks\\Welsh"
landmark_target_dir = "..\\data\\processed"

In [ ]:
# prepare dataset
